<a href="https://colab.research.google.com/github/ENGCSS/TRABALHO_Banco_de_Dados_Espaciais/blob/main/Banco_de_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Instala as bibliotecas necessárias
!pip install psycopg2 -q
!pip install ipython-sql -q

In [ ]:
#Carrega os pacotes necessários
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
# @title Configurando PostgreSQL no Google Colab

#Instala e inicia o PostgreSQL
!apt-get install -y postgresql postgresql-contrib > /dev/null 2>&1

#Ativa o serviço do PostgreSQL
!service postgresql start

#Configuração do usuário
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'curso_geo';"

## Cria o Banco de Dados e o conecta

In [ ]:
#Remove o banco de dados se ele já existir e cria um novo para evitar conflitos
!sudo -u postgres psql -c 'DROP DATABASE IF EXISTS bd_censo_ibge;'
!sudo -u postgres psql -c 'CREATE DATABASE bd_censo_ibge;'

#Define as credenciais como variáveis
dbname = 'bd_censo_ibge'  # Nome do banco de dados
user = 'postgres'         # Nome do usuário
password = 'curso_geo'    # Senha definida anteriormente
host = 'localhost'        # Endereço do host


#Conexão ao Banco de Dados
try:
    conn = psycopg2.connect(dbname=dbname, user=user, host=host, password=password)
    cur = conn.cursor()
    print("Conectado ao banco de dados com sucesso!")
except Exception as e:
    print("Erro ao conectar ao banco de dados: ", e)

## Conexão ao PostgreSQL com ipython-sql

In [ ]:
#Configura a string de conexão
connection_string = f"postgresql://{user}:{password}@{host}/{dbname}"

#Carrega a extensão ipython-sql
%load_ext sql
%sql $connection_string

#Executa uma consulta SQL
%sql SELECT version();

## Preparação e diagnóstico inicial do Banco de Dados

In [ ]:
#Consultar esquemas existentes
%%sql
SELECT schema_name
FROM information_schema.schemata;

In [ ]:
#Lista as tabelas no esquema ‘public’
%%sql
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';

In [ ]:
#Verifica a versão do Banco de Dados
%%sql
SELECT version();

In [ ]:
#Verifica o espaço em disco
%%sql
SELECT pg_size_pretty(pg_database_size(current_database()));

In [ ]:
#Verifica as conexões ativas
%%sql
SELECT *
FROM pg_stat_activity;

In [ ]:
#Consulta os índices existentes
%%sql
SELECT *
FROM pg_indexes
WHERE schemaname = 'public';

## Criação das tabelas

In [ ]:
%%sql
CREATE TABLE "municipio" (
  "cd_geocodi" int PRIMARY KEY,
  "nome_mun" varchar,
  "area_km2" float,
  "popul_2010" int,
  "popul_2022" int,
  "cod_rgi" int,
  "cod_uf" int
);

CREATE TABLE "regiao_imed" (
  "cod_rgi" int PRIMARY KEY,
  "nome_rgi" varchar,
  );

CREATE TABLE "estado" (
  "cod_uf" int PRIMARY KEY,
  "nome_uf" varchar,
  "sigla_uf" varchar,
  "cod_rg" int
);

CREATE TABLE "regiao" (
  "cod_rg" int PRIMARY KEY,
  "nome_rg" varchar
);

In [ ]:
#Verifica as Tabelas no Banco de Dados
%%sql
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';

In [ ]:
#Estabelece relações com chaves estrangeiras
%%sql
ALTER TABLE "municipio" ADD FOREIGN KEY ("cod_rgi") REFERENCES "regiao_imed" ("cod_rgi");
ALTER TABLE "municipio" ADD FOREIGN KEY ("cod_uf") REFERENCES "estado" ("cod_uf");
ALTER TABLE "estado" ADD FOREIGN KEY ("cod_rg") REFERENCES "regiao" ("cod_rg");

In [ ]:
#Carrega os dados das regiões
df_regiao = pd.read_excel('/content/Dados.xlsx',sheet_name='reg')
print(df_regiao.head())

#Carrega os dados dos estados
df_estado = pd.read_excel('/content/Dados.xlsx',sheet_name='est')
print(df_estado.head())

# Carrega os dados das regiões imediatas
df_regiao_imed = pd.read_excel('/content/Dados.xlsx',sheet_name='rgi')
print(df_regiao_imed.head())

# Carrega os dados populacionais dos municípios
df_municipio = pd.read_excel('/content/Dados.xlsx',sheet_name='mun')
print(df_municipio.head())

In [ ]:
#Verifica a tabela municipio
%%sql
SELECT * FROM regiao
LIMIT 10;

In [ ]:
# @title Inserção dos dados do DataFrame nas tabelas

# Cria o motor de conexão com o banco de dados
engine = create_engine(f'postgresql://{user}:{password}@{host}/{dbname}')

#-----------------------------------------------------------------------#

# Insere os dados na tabela regiao
df_regiao.to_sql('regiao', con=engine, if_exists='append', index=False)

# Insere os dados na tabela estado
df_estado.to_sql('estado', con=engine, if_exists='append', index=False)

# Insere os dados na tabela regiao_imed
df_regiao_imed.to_sql('regiao_imed', con=engine, if_exists='append', index=False)

# Insere os dados na tabela municipio
df_municipio.to_sql('municipio', con=engine, if_exists='append', index=False)

In [ ]:
%%sql
-- Verificando os dados inseridos em censo_mun
SELECT *
FROM municipio
LIMIT 10;

In [ ]:
#Adiciona índices ao esquema do banco de dados

%%sql
-- Índices para a tabela censo_mun
CREATE INDEX idx_mun_cod_rgi ON municipio USING btree (cod_rgi);
CREATE INDEX idx_mun_cod_uf ON municipio USING btree (cod_uf);

-- Índice para a tabela regiao_imed
CREATE INDEX idx_regiao_imed_cod_rgint ON regiao_imed USING btree (cod_rgint);

-- Índice para a tabela estado
CREATE INDEX idx_estado_cod_rg ON estado USING btree (cod_rg);

In [ ]:
%%sql
SELECT *
FROM municipio
ORDER BY nome_mun
LIMIT 10;

In [ ]:
%%sql
SELECT *
FROM municipio
WHERE cod_rgi=290030